# Investigating Embedding Drift for a Sentiment Classification Model

Imagine you're in charge of maintaining a model that takes as input online reviews of your U.S.-based product and classifies the sentiment of each review as positive, negative, or neutral. Your model initially performs well in production, but its performance gradually degrades over time.

Phoenix helps you surface the reason for this regression by analyzing the [embeddings](https://docs.arize.com/phoenix/concepts/embeddings) representing the text of each review. Your model was trained on English reviews, but as you'll discover, it's encountering Spanish reviews in production that it can't correctly classify.

In this tutorial, you will:
* Download curated datasets of embeddings and predictions
* Define a schema to describe the format of your data
* Launch Phoenix to visually explore your embeddings
* Investigate problematic clusters

Let's get started!

### 1. Install Dependencies and Import Libraries 📚

In [ ]:
%pip install -q arize-phoenix

In [ ]:
import pandas as pd
import phoenix as px

### 2. Download the Data 📊

Load your training and production data into two Pandas DataFrames.

In [ ]:
train_df = pd.read_parquet(
    "https://storage.googleapis.com/arize-assets/phoenix/datasets/unstructured/nlp/sentiment-classification-language-drift/sentiment_classification_language_drift_training.parquet",
)
prod_df = pd.read_parquet(
    "https://storage.googleapis.com/arize-assets/phoenix/datasets/unstructured/nlp/sentiment-classification-language-drift/sentiment_classification_language_drift_production.parquet",
)

Inspect a few rows of the training DataFrame.

In [ ]:
train_df.head()

The columns of the DataFrame are:
- **prediction_ts:** the Unix timestamps of your predictions
- **review_age**, **reviewer_gender**, **product_category**, **language:** the features of your model
- **text:** the text of each review
- **text_vector:** the embedding vectors representing each review
- **pred_label:** the label your model predicted
- **label:** the ground-truth label for each review

### 3. Launch Phoenix 🔥🐦

#### a) Define Your Schema

To launch Phoenix with your data, you first need to define a schema that tells Phoenix which columns of your DataFrames correspond to features, predictions, actuals (i.e., ground truth), embeddings, etc.

The trickiest part is defining embedding features. In this case, each embedding feature has two pieces of information: the embedding vector itself contained in the "text_vector" column and the review text contained in the "text" column.

In [ ]:
embedding_features = {
    "text_embedding": px.EmbeddingColumnNames(
        vector_column_name="text_vector", raw_data_column_name="text"
    ),
}
schema = px.Schema(
    timestamp_column_name="prediction_ts",
    prediction_label_column_name="pred_label",
    actual_label_column_name="label",
    embedding_feature_column_names=embedding_features,
)
schema


You'll notice that the schema above doesn't explicitly specify features. That's because feature columns are [implicitly inferred](https://docs.arize.com/phoenix/how-to/define-your-schema#implicit-features) if you don't pass `feature_column_names` to your `Schema` object.

#### b) Define Your Datasets 
Next, define your [primary and reference datasets](https://docs.arize.com/phoenix/concepts/phoenix-basics#which-dataset-is-which). In this case, your reference dataset contains training data and your primary dataset contains production data.

In [ ]:
prim_ds = px.Dataset(dataframe=prod_df, schema=schema, name="primary")
ref_ds = px.Dataset(dataframe=train_df, schema=schema, name="reference")

Inspect your primary dataset.

In [ ]:
prim_ds

#### c) Create a Phoenix Session

In [ ]:
session = px.launch_app(primary=prim_ds, reference=ref_ds)
session

#### d) Launch the Phoenix UI

You can open Phoenix by copying and pasting the output of `session.url` into a new browser tab.

In [ ]:
session.url

Alternatively, you can open the Phoenix UI in your notebook with

In [ ]:
session.view()

### 4. Explore Your Data 📈

Investigate troublesome clusters of your data:

1. Navigate to the "Embeddings" tab and click on "text_embedding".
2. In the Euclidean distance graph at the top of the page, click a point on the graph where the Euclidean distance is high.
3. Click on the top cluster in the panel on the left.
4. Use the panel at the bottom to examine the data points in this cluster.

Answer the questions below and click to see answers:

<details>
    <summary>
        What does the Euclidean distance graph measure?
    </summary>
    <p>
        This graph measures the drift of your production data relative to your training data over time. See <a href="https://docs.arize.com/phoenix/reference/metrics/euclidean-distance">here</a> for details.
    </p>
</details>

<details>
    <summary>
        What do the points in the point cloud represent?
    </summary>
    <p>
        Each point in the point cloud corresponds to a single product review. Phoenix has taken the high-dimensional embeddings in your original DataFrame and has reduced the dimensionality so that you can view them in lower dimensions.
    </p>
</details>

<details>
    <summary>
        What do you notice about the cluster you selected?
    </summary>
    <p>
        It consists almost entirely of production data, meaning that your model is seeing data in production the likes of which it never saw during training.
    </p>
</details>

<details>
    <summary>
        What's gone wrong with your model in production?
    </summary>
    <p>
        Your model was fine-tuned on examples of labeled product reviews in English. In production, your model is encountering product reviews in Spanish whose sentiment it cannot correctly predict.
    </p>
</details>

Congrats! You've identified the root cause of your model's performance issue.

### 5. Close the App 🧹

When you're done, don't forget to close the app.

In [ ]:
px.close_app()